### Imports

In [1]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt
from tip_finding.tip_finding import tip_finder_decimation

### Loading in data

In [3]:
load = pd.read_csv("endpoints_gt4.csv")
endpoints_dict = {}

In [4]:
load.head()

,neuron,ng_link,seg_id,pink_pts,num_endpoints,endpoints,comments,detailed_comments
0,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"((402584, 228856, 23991), (402985, 229235, 235...",good,NaN
1,NaN,NaN,864691135247440303,"(401258, 224832, 24029)",3,"((401612, 224623, 23991), (405257, 226318, 236...",good,NaN
2,NaN,NaN,864691134794123793,"(401314, 228366, 24424)",2,"((401242, 228382, 24444), (400982, 228457, 245...",m,merged to two axon pieces
3,NaN,NaN,864691135772363453,"(400199, 220721, 24029)",7,"((401289, 218721, 23991), (399895, 216533, 235...",m,NaN
4,NaN,NaN,864691135314714227,"(397870, 232292, 24004)",2,"((397867, 232230, 23999), (397854, 232252, 239...",good,NaN


In [5]:
#creating series objects for the desired collumns 
segIDs = load['seg_id']
Endpoints = load['endpoints']

In [6]:
# def endpoint_generator(load, invalidation_d, humfrey_iters, decimation_factor):
for idx, i in enumerate(load['seg_id']):
    if idx==10:
        break
    try:
        t1, skel, mesh_obj = tip_finder_decimation(str(i))
        endpoints_dict[i] = t1
    except:
        print(f"\n\nSeg {i} returned error on get. Skipping.\n")
        pass

load['endpoints'] = load['endpoints'].apply(lambda x: list(ast.literal_eval(x)))
load['proposed_endpoints'] = load.seg_id.map(endpoints_dict)

07/26/2022 02:59:35 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 3996/3996 [00:00<00:00, 258640.76it/s]
07/26/2022 02:59:42 PM WARNING: face_normals incorrect shape, ignoring! 


  0%|          | 0/10649 [00:00<?, ?it/s]/Users/tinaxie/Desktop/agt/campfire/venv/lib/python3.8/site-packages/meshparty/skeletonize.py:613: RuntimeWarning: invalid value encountered in multiply
  target = np.nanargmax(root_ds * valid)
100%|██████████| 10649/10649 [00:00<00:00, 125843.54it/s]
07/26/2022 02:59:46 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 1140/1140 [00:00<00:00, 172468.13it/s]
07/26/2022 02:59:54 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 91376/91376 [00:01<00:00, 75518.99it/s] 
07/26/2022 03:00:15 PM WARNING: face_normals incorrect shape, ignoring! 


...



Seg 864691133946526057 returned error on get. Skipping.



07/26/2022 03:01:15 PM WARNING: face_normals incorrect shape, ignoring! 




Seg 864691133712131152 returned error on get. Skipping.



07/26/2022 03:01:16 PM WARNING: face_normals incorrect shape, ignoring! 


100%|██████████| 204/204 [00:00<00:00, 75247.39it/s]


In [7]:
#correctly  have 2 collumns for the proposed vs gt endpoints
load.head(10)

,neuron,ng_link,seg_id,pink_pts,num_endpoints,endpoints,comments,detailed_comments,proposed_endpoints
0,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"[(402584, 228856, 23991), (402985, 229235, 235...",good,NaN,"[(402990, 229222, 23552), (402567, 228758, 239..."
1,NaN,NaN,864691135247440303,"(401258, 224832, 24029)",3,"[(401612, 224623, 23991), (405257, 226318, 236...",good,NaN,"[(404862, 225350, 23698), (409143, 219355, 238..."
2,NaN,NaN,864691134794123793,"(401314, 228366, 24424)",2,"[(401242, 228382, 24444), (400982, 228457, 245...",m,merged to two axon pieces,"[(400269, 228563, 24514), (400319, 228912, 245..."
3,NaN,NaN,864691135772363453,"(400199, 220721, 24029)",7,"[(401289, 218721, 23991), (399895, 216533, 235...",m,NaN,"[(399932, 216516, 23518), (370591, 192198, 219..."
4,NaN,NaN,864691135314714227,"(397870, 232292, 24004)",2,"[(397867, 232230, 23999), (397854, 232252, 239...",good,NaN,"[(397931, 232013, 24000), (397815, 232393, 239..."
5,NaN,NaN,864691135552841025,"(403272, 227529, 24394)",2,"[(403332, 227545, 24398), (403566, 227614, 244...",good,NaN,"[(403709, 227788, 24419), (403094, 227479, 244..."
6,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135516937065,"(77233, 113188, 20454)",2,"[(77262, 113088, 20442), (79178, 109116, 20402)]",good,NaN,"[(78850, 110705, 20424), (77219, 113151, 20441..."
7,NaN,NaN,864691134406233920,"(100382, 143836, 21642)",2,"[(100475, 143982, 21654), (100583, 144194, 217...",good,NaN,"[(100979, 144058, 21663), (100780, 144830, 216..."
8,NaN,NaN,864691136080776044,"(80699, 146603, 20732)",5,"[(80890, 147158, 20708), (81592, 148549, 20453...",mm,NaN,"[(83532, 155279, 20488), (81462, 151688, 20595..."
9,NaN,NaN,864691134330810585,"(72020, 146120, 20454)",2,"[(71898, 146194, 20442), (71499, 146291, 20404)]",good,NaN,"[(71940, 146124, 20442), (71395, 146316, 20403)]"


### Helper functions we wrote

In [8]:

def find_euclidian_distance(proposed_endpoint, gt_endponts_array):
    smallest_distance = 2**30
    for endpoint in gt_endponts_array:
        difference = proposed_endpoint - endpoint
        distance = np.sqrt(np.sum(np.square(difference)))
        if(distance < smallest_distance):
            smallest_distance = distance 
    return smallest_distance 
            
    #could add endpoint to dict

In [9]:
def get_recall(num_correct, array_endpoints): 
    recall = num_correct / len(array_endpoints)
    return recall * 100

In [10]:
def get_precision(num_correct, array_proposed): 
    precision = num_correct / len(array_proposed)
    return precision * 100

### Analysis functions

In [11]:
"""
Various utility classes and functions for Confirms.
"""
from copy import deepcopy

import numpy as np
import pandas as pd
from scipy import spatial
from scipy.optimize import linear_sum_assignment


def calculate_precision_recall(tp, fp, fn):
    """
    Calculate precision/recall from given true positives, false positives, and false negatives.
    Parameters
    ----------
    tp : int
        The number of true positives.
    fp : int
        The number of false positives.
    fn : int
        The number of false negatives.
    Returns
    -------
    precision : float
        The precision score.
    recall : float
        The recall score.
    """
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    return precision, recall


def calculate_f1(precision, recall):
    """
    Calculate the F1 score from precision/recall scores.
    Parameters
    ----------
    precision : float
        The precision score.
    recall : float
        The recall score.
    Returns
    -------
    f1 : float
        The F1 score.
    """
    return 2 * ((precision * recall) / (precision + recall))


def get_summary_metrics(array):
    """
    Calculate a number of summary metrics on an array of numbers.
    Parameters
    ----------
    array : array_like
        Array containing numbers who summary metrics is desired.
    Returns
    -------
    metrics : dict
        Dict containing the mean, median, max, min, range, standard deviation,
        and variance of the input array.
    """
    summary_object = {}
    array = np.array(array)

    summary_object["mean"] = np.mean(array)
    summary_object["median"] = np.median(array)
    summary_object["max"] = np.amax(array)
    summary_object["min"] = np.amin(array)
    summary_object["range"] = np.mean(array)
    summary_object["stddev"] = np.std(array)
    summary_object["variance"] = np.var(array)

    return summary_object


def munkres_assignment(workers, jobs):
    """
    Perform hungarian-munkres assignment.
    Parameters
    ----------
    workers : array_like
        Array containing the first set of points.
    jobs : array_like
        Array containing the second set of points.
    Returns
    -------
    cost_matrix : numpy.ndarray
        Matrix containing pairwise distances (the cost of assignment).
    row_ind : numpy.ndarray
        Row indices of cost_matrix for optimal assignment.
    col_ind : numpy.ndarray
        Column indices of cost_matrx for optimal assignment.
    """
    cost_matrix = spatial.distance.cdist(workers, jobs, "euclidean")
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return cost_matrix, row_ind, col_ind


def make_isotropic(xyz, correction, dimen=2):
    """
    Correct anisotropy in a collection of x, y, z coordinates.
    This function performs a deepcopy of xyz before making the necessary modifications.
    Parameters
    ----------
    xyz : numpy.ndarray or pandas.DataFrame:
        The coordinate values.
    correction : float
        The value to correct anisotrophy.
    dimen : int
        Index of last dimension to which to apply the correction. Default is 2.
    Returns
    -------
    iso_xyz : numpy.ndarray or pandas.DataFrame
        An isotropic version of xyz.
    """

    if not isinstance(xyz, np.ndarray) and not isinstance(xyz, pd.DataFrame):
        xyz = np.asarray(xyz)

    shape = np.shape(xyz)
    isotropic_xyz = xyz.copy()
    size = shape[-1]
    if dimen >= size or dimen < 0:
        raise ValueError("improper dimen value (valid: 0 through {})".format(size - 1))
    if isinstance(xyz, np.ndarray):
        isotropic_xyz[..., dimen] = isotropic_xyz[..., dimen] * correction
    else:
        isotropic_xyz.iloc[:, dimen] = isotropic_xyz.iloc[:, dimen] * correction
    return isotropic_xyz


# def get_range(center, pixel_size, radius):
#     """
#     <Description here>
#     Parameters
#     ----------
#     center : <type>
#         <Description>
#     pixel_size : <type>
#         <Description>
#     radius : <type>
#         <Description>
    
#     Returns
#     -------
#     range : numpy.ndarray
#         <Description>
#     """
#     return np.asarray(
#         [int(center - (radius / pixel_size)), int(center + (radius / pixel_size))],
#         dtype="int",

In [12]:
"""
Confirms synapse processing and analysis functions.
"""
from collections import namedtuple

import numpy as np
import pandas as pd
from scipy import spatial
from scipy.optimize import linear_sum_assignment

# from . import utils
# import utils

SynapseMetrics = namedtuple(
    "SynapseMetrics",
    ["precision", "recall", "f1", "tp_gt_ids", "tp_test_ids", "fp_ids", "fn_ids"],
)


def filter_synapse_id_core(volume, xyz, ids, box_radius_nm=2500):
    """
    Filter synapses to only return those within a central core.
    Parameters
    ----------
    volume : dict
        The volume to filter on.
    xyz : array_like
        Synapse xyz coordinates.
    ids : array_like
        Synapse hash (ids).
    box_radius_nm : int
        radius of cube, from volume core.
    Returns
    -------
        xyz : numpy.ndarray
            Synapse coordinates.
        ids : numpy.ndarray
            Synapse ids.
    """

    xyz_out = []
    id_out = []
    center = np.asarray(volume["center"], "float")
    base_resolution = np.asarray(volume["base_resolution"], "float")
    annotation_resolution = np.asarray(volume["resolution"], "float")

    pad_vx = box_radius_nm / base_resolution[0] / (2 ** annotation_resolution)
    pad_vy = box_radius_nm / base_resolution[1] / (2 ** annotation_resolution)
    pad_vz = box_radius_nm / base_resolution[2]
    xr = (center[0] - pad_vx, center[0] + pad_vx)
    yr = (center[1] - pad_vy, center[1] + pad_vy)
    zr = (center[2] - pad_vz, center[2] + pad_vz)

    for i in range(len(xyz)):
        x = xyz[i][0]
        y = xyz[i][1]
        z = xyz[i][2]

        if (
            x > xr[0]
            and x < xr[1]
            and y > yr[0]
            and y < yr[1]
            and z > zr[0]
            and z < zr[1]
        ):
            xyz_out.append(xyz[i])
            id_out.append(ids[i])

    return np.array(xyz_out), np.array(id_out, dtype=np.object)


def synapse_match(
    xyz_truth, xyz_detect, id_truth, id_detect, thresh
):  # pylint: disable=R0914
    """
    <Description here>
    Parameters
    ----------
    xyz_truth : array_like
        <description>
    xyz_detect : array_like
        <description>
    id_truth : array_like
        <description>
    id_detech : array_like
        <description>
    thresh : float
        <description>
    Returns
    -------
    id_lookup : <type>
        <description>
    """

    # pylint: disable=C0103

    # Ensure we have numpy arrays
    xyz_truth = np.asarray(xyz_truth)
    xyz_detect = np.asarray(xyz_detect)
    id_truth = np.asarray(id_truth)
    id_detect = np.asarray(id_detect)

    cost, row_ind, col_ind = munkres_assignment(xyz_truth, xyz_detect)
    match_idx = np.where(cost[row_ind, col_ind] < thresh)

    if len(match_idx) > 0:
        # row is idx of GT TP
        # col is idx of student TP
        gt_tp_idx, det_tp_idx = row_ind[match_idx], col_ind[match_idx]
        gt_tp_ids, det_tp_ids = id_truth[gt_tp_idx], id_detect[det_tp_idx]
        # Combine into pairs
        id_lookup = np.column_stack((gt_tp_ids, det_tp_ids))
    else:
        gt_tp_idx = []
        det_tp_idx = []
        id_lookup = np.column_stack(
            (np.array([], dtype="object"), np.array([], dtype="object"))
        )

    # not in row (set diff) are FN
    gt_syn_idx = np.arange(0, len(xyz_truth))
    fn_idx = np.setdiff1d(gt_syn_idx, gt_tp_idx)
    if len(fn_idx) > 0:
        fn_ids = id_truth[fn_idx]
        id_lookup_fn = np.column_stack((fn_ids, np.repeat(None, len(fn_ids))))
        id_lookup = np.concatenate((id_lookup, id_lookup_fn))

    # not in col (set diff) are FP
    det_syn_idx = set(np.arange(0, len(xyz_detect)))
    fp_idx = np.asarray(list(det_syn_idx.difference(det_tp_idx)))
    if len(fp_idx) > 0:
        fp_ids = id_detect[fp_idx]
        id_lookup_fp = np.column_stack((np.repeat(None, len(fp_ids)), fp_ids))
        id_lookup = np.concatenate((id_lookup, id_lookup_fp))

    return pd.DataFrame(id_lookup, columns=["ground_truth", "detect"])


def run_synapse_analysis(
    gt_xyzs,
    gt_ids,
    test_xyzs,
    test_ids,
    threshold,
    iso_corrected=False,
    iso_correction=1,
):
    """
    <Description here>
    Parameters
    ----------
    gt_xyzs : numpy.ndarray
        Array of ground truth xyz coordinates
    gt_ids : numpy.ndarray
        Array of ids associated with ground truth xyz coordinates
    test_xyzs : numpy.ndarray
        Array of test xyz coordinates
    test_ids : numpy.ndarray
        Array of ids associated with test xyz coordinates
    threshold : float
        Synapse matching threshold
    iso_corrected : boolean
        Mark whether the data is isotropic. If not, it will be made isotropic using the
        `iso_correction` parameter.
    iso_correction : float
        Value to correct anistropy.
    Returns
    -------
    sm : SynapseMetrics
        Resultant object containing precision, recall, and F1 scores, along with
        with true positive (both ground truth and test), false positive, and false negative
        ids.
    """
    # pylint: disable=R0913,R0914
    if not iso_corrected:
        # gt_xyzs = utils.make_isotropic(gt_xyzs, iso_correction)
        gt_xyzs = make_isotropic(gt_xyzs, iso_correction)
        # test_xyzs = utils.make_isotropic(test_xyzs, iso_correction)
        test_xyzs = make_isotropic(test_xyzs, iso_correction)

    results_table = synapse_match(gt_xyzs, test_xyzs, gt_ids, test_ids, threshold)

    tp = results_table.dropna()
    fn = results_table[results_table.detect.isnull()]
    fp = results_table[results_table.ground_truth.isnull()]

    assert len(tp.ground_truth) == len(
        tp.detect
    ), "true positive ground truth and test size mismatch"

    tp_count = len(tp)
    fp_count = len(fp)
    fn_count = len(fn)

    try:
        # precision, recall = utils.calculate_precision_recall(
        #     tp_count, fp_count, fn_count
        # )

        precision, recall = calculate_precision_recall(
            tp_count, fp_count, fn_count
        )

    except ZeroDivisionError:
        precision, recall = np.nan, np.nan

    try:
        # f1 = utils.calculate_f1(precision, recall)
        f1 = calculate_f1(precision, recall)
    except ZeroDivisionError:
        f1 = np.nan

    sm = SynapseMetrics(
        precision=precision,
        recall=recall,
        f1=f1,
        tp_gt_ids=np.asarray(tp.ground_truth),
        tp_test_ids=np.asarray(tp.detect),
        fp_ids=np.asarray(fp.detect),
        fn_ids=np.asarray(fn.ground_truth),
    )
    return sm

In [13]:
#TESTING
gt_xyzs = np.array([[167695,211664,20725],
                    [145921,225039,22976]])

gt_ids = np.array([864691135537360703, 864691135537360703])

test_xyzs = np.array([[145718,224993,22957],
                      [147843,224023,22885],
                      [152766,222078,22418],
                      [166663,211491,20783]])

test_ids = np.array(
    [864691135537360703, 864691135537360703, 864691135537360703, 864691135537360703])

threshold = 500

analysis = run_synapse_analysis(
    gt_xyzs,
    gt_ids,
    test_xyzs,
    test_ids,
    threshold,
    iso_correction=10,
)

### Using analysis functions and helper functions to calculate recall, precision, f1 for multiple seg ids

In [18]:
out_pd = load[['seg_id', 'num_endpoints', 'endpoints', 'comments', 'proposed_endpoints']]


In [20]:
out_pd.head(2)

,seg_id,num_endpoints,endpoints,comments,proposed_endpoints
0,864691135909994000,2,"[(402584, 228856, 23991), (402985, 229235, 235...",good,"[(402990, 229222, 23552), (402567, 228758, 239..."
1,864691135247440303,3,"[(401612, 224623, 23991), (405257, 226318, 236...",good,"[(404862, 225350, 23698), (409143, 219355, 238..."


In [15]:
#TEST 1: invalidation_d at default: 12000, WITH Humfrey_Smoothing: 50 iterations, Decimation_Factor = 0.50

#should track total amount of correct gt recovered 

for index, row in load.iterrows():
    #reset num correct
    num_correct = 0
    threshold = 500
    endpoint_array = np.array(row["endpoints"])
    proposed_endpoints_array = np.array(row["proposed_endpoints"])
    print(f"Seg id: {row['seg_id']}")
    segID = row["seg_id"]
    print("Endpoint Array:")
    print(endpoint_array)
    #print(len(endpoint_array))
    print()
    print("Proposed Endpoint Array:")
    print(proposed_endpoints_array)
    print()
    #print(load["Endpoints"])
    
    #skip anything seg_id that isn't 'good'
    if row["comments"] != 'good':
        print("skipping \n")
        continue

    #learn better pandas filtered rather than if else statements     
    #maybe skip v-shaped segments
    #add in a label for ground truth 
    
    #if both say no endpoints, then it's correct
    if (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) == 0):
        print("correct, no endpoints")
        print("\nNEXT POINTS")
        continue
    #if we propose no endpoints but there are endpoints, it's wrong
    elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) > 0):
        print("no proposed endpoints found, but ground truth endpoints exist")
        print("\nNEXT POINTS")
        continue 

    #if we propose endpoints but there are none, it's wrong
    elif proposed_endpoints_array.size > 0 and endpoint_array.size == 0:
        print("not correct--too many proposed endpoints but none ground truth endpoints exist")
        print("\nNEXT POINTS")
        continue
    #should skip for now if the endpoint array is size zero to avoid divide by zero error
    elif endpoint_array.size == 0:
        print("array size is 0")
        print("\nNEXT POINTS")
        continue
    else:                                                                                   
        for proposed_endpoint in proposed_endpoints_array:
            smallest_distance= find_euclidian_distance(proposed_endpoint, endpoint_array)
            print(smallest_distance)
            
            #compare to threshold and return yes or no 
            if smallest_distance < threshold:
                print("correct")
                num_correct = num_correct + 1
                print()
            else:
                print("wrong")
                print()

    endpoint_ids = []
    test_ids = []
    for i in range(0, len(endpoint_array)):
        endpoint_ids.append(segID)
        
    for i in range(0, len(proposed_endpoints_array)):
        test_ids.append(segID)

    #get precision and recall at end of each iteration 
    analysis = run_synapse_analysis(
        endpoint_array,
        np.array(endpoint_ids),
        proposed_endpoints_array,
        np.array(test_ids),
        threshold,
        iso_correction=10,
        )
    print(analysis)
    print()
    print("NEXT SEGID: ")
    print()

        # recall = get_recall(num_correct, endpoint_array)
        # precision = get_precision(num_correct, proposed_endpoints_array)
        # print("recall = " + str(recall))
        # print("precision = " + str(precision))
        # if recall > 100.0 or precision > 100.0:
        #     print("ERROR: RESULTS OVER 100% DETECTED")
        # print()
        # print("\nNEXT POINTS")
        
        
    

Seg id: 864691135909994000
Endpoint Array:
[[402584 228856  23991]
 [402985 229235  23554]]

Proposed Endpoint Array:
[[402990 229222  23552]
 [402567 228758  23991]]

14.071247279470288
correct

99.46356116689167
correct

SynapseMetrics(precision=1.0, recall=1.0, f1=1.0, tp_gt_ids=array([864691135909994000, 864691135909994000]), tp_test_ids=array([864691135909994000, 864691135909994000]), fp_ids=array([], dtype=int64), fn_ids=array([], dtype=int64))

NEXT SEGID: 

Seg id: 864691135247440303
Endpoint Array:
[[401612 224623  23991]
 [405257 226318  23620]
 [409162 219642  23891]]

Proposed Endpoint Array:
[[404862 225350  23698]
 [409143 219355  23868]
 [405021 226754  23620]
 [401506 224664  23990]]

1048.3954406615856
wrong

288.54635676091976
correct

495.77414212522217
correct

113.65737987477979
correct

SynapseMetrics(precision=0.75, recall=1.0, f1=0.8571428571428571, tp_gt_ids=array([864691135247440303, 864691135247440303, 864691135247440303],
      dtype=object), tp_test_ids=arr

make threshold for each endpoint
experiment with box (cannot overlap)
for each endpoint see if it finds in box -- if yes true found endpont, if false --not an endpoint
find distance from predicted to ground truth 
could still do MSE 

Threshold: 
-find the size of the segment (maybe connected componenets)
-array from the skel
-flat is edge case 
-maybe a percentage 
--a hard number 





### Comments

Need to complie results!!!! 

add to dataframe all of the precision and recalls 
plot the results and check for 
params don't have to be the best 


make a sepereate dataframe 

look: pd.concat

mention the params 

or have a collumn that has run number 

PRECISION: #test at ground truth / # test  

RECALL: # gt recovered / number ground truth 
(would be 100% if you found all the correct endponts...NO EXTRAS)